In [19]:
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO
import pandas as pd
from Bio.Seq import Seq
from typing import List, Dict, Union
from teemi.design.combinatorial_design import simple_amplicon_maker
from pydna.design import primer_design
from pydna.dseqrecord import Dseqrecord


def open_gff3_files(path: str = "") -> List[List[str]]:
    """
    Opens and reads a GFF3 file and returns its contents as a list of lists.

    Parameters:
    -----------
    path: str
        The path to the GFF3 file.

    Returns:
    --------
    List[List[str]]
        A list of lists containing the contents of the GFF3 file.
    """
    with open(path, "r") as infile:
        LINES = []
        for line in infile:
            LINES.append(line[:].split("\t"))
        LINES = LINES[1:]

    return LINES


def tidy_up_gff(lst_of_gff: list) -> list:
    """
    This function takes a list of GFF lines and returns a list of dictionaries,
    with each dictionary containing information on the signal peptides in the GFF file.

    Parameters:
    lst_of_gff (list): A list of GFF lines.

    Returns:
    list_of_peptides (list): A list of dictionaries, with each dictionary containing information on the signal peptides in the GFF file.
    """
    signal_peptides = {}
    list_of_peptides = []

    for peptide in lst_of_gff:
        signal_peptides["gene"] = peptide[0][:19]
        signal_peptides["start_pos"] = int(peptide[3]) - 1
        signal_peptides["end_pos"] = int(peptide[4]) + 1
        signal_peptides["signal_peptide_likelyhood"] = peptide[5]
        list_of_peptides.append(signal_peptides)
        signal_peptides = {
            "gene": "",
            "start_pos": "",
            "end_pos": "",
            "signal_peptide_likelyhood": "",
        }

    return list_of_peptides


def dict_of_signal_peptides(path: str = "") -> List[Dict[str, Union[str, int]]]:
    """
    Given a path to a GFF3 file, returns a list of dictionaries with information on signal peptides.

    Args:
        path (str): Path to the GFF3 file. Default is an empty string.

    Returns:
        list: A list of dictionaries where each dictionary contains the following keys:
            - 'gene' (str): Gene name of the signal peptide.
            - 'start_pos' (int): Start position of the signal peptide in the protein sequence.
            - 'end_pos' (int): End position of the signal peptide in the protein sequence.
            - 'signal_peptide_likelyhood' (str): The likelihood of the sequence being a signal peptide.
    """
    gff = open_gff3_files(path)
    dict_of_signal_peptides = tidy_up_gff(gff)
    return dict_of_signal_peptides


def read_gff_to_pd(path: str = "") -> pd.DataFrame:
    """
    Reads a GFF3 file and returns a pandas DataFrame with columns 'gene', 'start_pos', 'end_pos',
    and 'signal_peptide_likelyhood'.

    Parameters:
    -----------
    path : str
        The path to the GFF3 file.

    Returns:
    --------
    df : pandas.DataFrame
        A DataFrame with columns 'gene', 'start_pos', 'end_pos', and 'signal_peptide_likelyhood'.
    """

    gff = open_gff3_files(path)
    dict_of_signal_peptides = tidy_up_gff(gff)
    df = pd.DataFrame.from_records(dict_of_signal_peptides)

    return df


import requests
import json


def primer_ta_neb(primer1, primer2, conc=0.4, prodcode="phusion-0"):
    """Calculates primer pair melting temp TA,  from NEB.

    Parameters
    ----------
    primer1 : str
        first primer to be used for finding the optimal ta
    primer2 : str
        second primer to be used for finding the optimal ta
    conc : float
    prodcode : str
        find product codes on nebswebsite: https://tmapi.neb.com/docs/productcodes

    Returns
    -------
    ta : int
        primer pair annealing temp

    """

    url = "https://tmapi.neb.com/tm/batch"
    seqpairs = [[primer1, primer2]]

    input = {"seqpairs": seqpairs, "conc": conc, "prodcode": prodcode}
    headers = {"content-type": "application/json"}
    res = requests.post(url, data=json.dumps(input), headers=headers)

    r = json.loads(res.content)

    if r["success"]:
        for row in r["data"]:
            return row["ta"]

    else:
        print("request failed")
        print(r["error"][0])


def primer_tm_neb1(primer, conc=0.4, prodcode="phusion-0"):
    """Calculates a single primers melting temp from NEB.

    Parameters
    ----------
    primer1 : str
    conc : float
    prodcode : str
        find product codes on nebswebsite: https://tmapi.neb.com/docs/productcodes

    Returns
    -------
    tm : int
        primer melting temperature

    """

    url = "https://tmapi.neb.com/tm/batch"
    seqpairs = [[primer]]

    input = {"seqpairs": seqpairs, "conc": conc, "prodcode": prodcode}
    headers = {"content-type": "application/json"}
    res = requests.post(url, data=json.dumps(input), headers=headers)

    r = json.loads(res.content)

    if r["success"]:
        for row in r["data"]:
            return row["tm1"]
    else:
        print("request failed")
        print(r["error"][0])
        
        
def make_amplicons(
    list_of_amplicons: list, target_tm=58, limit=10, tm_function=primer_tm_neb1):
    """Generates pydna.amplicons which contains primers with a target temperature.

    Parameters
    ----------
    list_of_amplicons : list
        list of pydna.Dseqrecords
    target_tm : int
        representing the target melting temperature for the primers (default=55)
    limit: int
        representing the minimum primer size (default=5)
    tm_function : function
        for calculating primer melting temperature (default=primer_tm_neb)

    Returns:
    amplicons: list
        list of amplicon objects with designed primer sequences
    """
    amplicons = []
    for i in range(len(list_of_amplicons)):
        amplicon = primer_design(
            list_of_amplicons[i],
            target_tm=target_tm,
            limit=limit,
            tm_function=tm_function,
        )

        amplicons.append(amplicon)

    return amplicons


# Primer design notebook

## Introdution: 

What you will need: 
1. Sequences that you want to amplify
2. SignalP output for those sequences


Steps:

1) Read sequences from a text file
2) Upload and Process sigP output
3) Remove SP from nucleotide sequence
4) Make primers



Note: if you want to change primer_calculation parameters check the functions above. 

## 1. Read sequences from a text file

In [20]:
from Bio import SeqIO

In [21]:
sequences = list()
for sequence in SeqIO.parse('../data/external/GeneByLocusTag.fasta',format='fasta'):
    print(sequence)
    sequences.append(sequence)

ID: AO090005001389
Name: AO090005001389
Description: AO090005001389
Number of features: 0
Seq('ATGGGAGCTCCCCGGCTTCGGATCAAAGGCGCAACCTTCAAAGACCCAAATAAT...TAG')
ID: AO090038000444
Name: AO090038000444
Description: AO090038000444
Number of features: 0
Seq('ATGCGTTCCTTGTCGTCTATCGCACTCCTGTCTGTGGTGGGAGCTGCGTCTGCC...TAA')
ID: AO090012000917
Name: AO090012000917
Description: AO090012000917
Number of features: 0
Seq('ATGCGGGCGTCACTACCGTTTCTTACTGCCTTGGGATGTATTCCAGCCGCTTTG...TGA')
ID: AO090005001553
Name: AO090005001553
Description: AO090005001553
Number of features: 0
Seq('ATGAAATTCCGTAACCTTTTCTTTGCTGCCGTAGCTGGCTCTGCGGTTGCTGCT...TAG')
ID: AO090011000715
Name: AO090011000715
Description: AO090011000715
Number of features: 0
Seq('ATGAGAATCGGCAACTTGATCGTGGCTGCAAGTGCTGCAAGCCTGGTGCATGCG...TAA')
ID: AO090011000757
Name: AO090011000757
Description: AO090011000757
Number of features: 0
Seq('ATGAAAGTCACCAGGCTAGCGGTTCTGAATACCCTGGCAACCCTAACTGTTGCC...TAG')
ID: AO090003000990
Name: AO090003000990
Description:

## 2. Import sigP results

In [23]:
signal_pep = read_gff_to_pd('../data/external/SigP_results/test_3/output.gff3')
signal_pep

,gene,start_pos,end_pos,signal_peptide_likelyhood
0,AO090038000444,0,19,0.9994235
1,AO090012000917,0,20,0.99983215
2,AO090005001553,0,28,0.999823
3,AO090011000715,0,28,0.9998376
4,AO090011000757,0,19,0.9998413
5,AO090003000990,0,23,0.9998205
6,AO090012000006,0,27,0.99984413
7,AO090003001590,0,19,0.99982584


In [24]:
N_pos = signal_pep['end_pos'].to_list()
genes_with_signal_peptides = signal_pep['gene'].to_list()

my_dict = {'name':genes_with_signal_peptides, 'n_pos':N_pos }
my_dict

{'name': ['AO090038000444',
  'AO090012000917',
  'AO090005001553',
  'AO090011000715',
  'AO090011000757',
  'AO090003000990',
  'AO090012000006',
  'AO090003001590'],
 'n_pos': [19, 20, 28, 28, 19, 23, 27, 19]}

# 3. Remove SP from nucleotide sequence

In [25]:
clean_seq = list()
for i in range(len(genes_with_signal_peptides)):
    for seq in sequences: 
        if genes_with_signal_peptides[i] == seq.id:
            clean_seq.append(seq[N_pos[i]*3:-3])
            
for seq in sequences:
    if seq.id not in genes_with_signal_peptides:
        clean_seq.append(seq[3:-3])
            
len(clean_seq)

12

In [26]:
clean_seq

[SeqRecord(seq=Seq('GCTGGTCCTTGGGCTCAGTGTGGAGGCAAGTCCTTCTCCGGCTCATCCGAGTGT...AAC'), id='AO090038000444', name='AO090038000444', description='AO090038000444', dbxrefs=[]),
 SeqRecord(seq=Seq('CCCCATCCCCGAGTGCAGAGTCCGGAGTATGTCAACTGGACAACTTTCAAAGCC...AAG'), id='AO090012000917', name='AO090012000917', description='AO090012000917', dbxrefs=[]),
 SeqRecord(seq=Seq('TCAGTTTTCCAATGTTAGTCTGCCCTTCTTCTTCTTCTTCTTCTTCTTCTTTCT...GGG'), id='AO090005001553', name='AO090005001553', description='AO090005001553', dbxrefs=[]),
 SeqRecord(seq=Seq('TCGGGATTCACCTGTACGTCAAGTATTGGCTCTGAAGTTACTGGATCTAACGAA...CTG'), id='AO090011000715', name='AO090011000715', description='AO090011000715', dbxrefs=[]),
 SeqRecord(seq=Seq('CTTCCAACAACCGACAAGACAATCACCTCTAGTAATGGAACCGATCTCTTCAAA...AGC'), id='AO090011000757', name='AO090011000757', description='AO090011000757', dbxrefs=[]),
 SeqRecord(seq=Seq('ACTATGTCAGTATAACCGTGATGTTACCCCTTTTGCTCTGCATAGTCCCTTACT...TGC'), id='AO090003000990', name='AO090003000990', description='AO09

In [27]:
clean_seq = [Dseqrecord(seq) for seq in clean_seq]

In [28]:
clean_seq[0].seq

Dseq(-1510)
GCTG..GAAC
CGAC..CTTG

## 4. Make primers

In [29]:
amplicons = make_amplicons(clean_seq,
                           target_tm=60, # targtet temp
                           limit=15,  # min length of primers
                           tm_function = primer_tm_neb1)

In [30]:
amplicons[0].forward_primer.seq

Seq('GCTGGTCCTTGGGCT')

In [31]:
forward_primer = [str(f.forward_primer.seq) for f in amplicons]
r_primer = [str(r.reverse_primer.seq) for r in amplicons]
name = [str(r.name) for r in amplicons]
gene = [str(r.id) for r in clean_seq]
aneal_f = [primer_tm_neb1(str(r)) for r in forward_primer]
aneal_r = [primer_tm_neb1(str(r)) for r in r_primer]

In [32]:
ta= [primer_ta_neb(str(f.forward_primer.seq),str(f.reverse_primer.seq))  for f in amplicons]

In [34]:
df = pd.DataFrame({'template': gene, 'f_primer':forward_primer, 'r_primer':r_primer, 'f_tm': aneal_f, 'r_tm': aneal_r, 'ta':ta})

In [35]:
df

,template,f_primer,r_primer,f_tm,r_tm,ta
0,AO090038000444,GCTGGTCCTTGGGCT,GTTCTTGTTGACATTTTCCATATGGTC,57,60,61
1,AO090012000917,CCCCATCCCCGAGTG,CTTCGTATCATGGAATCTCGTAAGGTC,57,62,61
2,AO090005001553,TCAGTTTTCCAATGTTAGTCTGC,CCCCAGGTATGCCTG,59,54,58
3,AO090011000715,TCGGGATTCACCTGTAC,CAGGCACTGGCTGTA,54,54,58
4,AO090011000757,CTTCCAACAACCGACAAGA,GCTGCTTCGACCTTTACA,57,56,60
5,AO090003000990,ACTATGTCAGTATAACCGTGATG,GCACTGCTTAGGGAACTC,57,57,60
6,AO090012000006,GATGCCGAGGGTTGG,TTTACGAGTGATTGTGATAGTGATGT,55,60,59
7,AO090003001590,AGTGCCAATTGTACAGGG,AACAGTCAAAGTAAAGTTGACCG,56,59,60
8,AO090005001389,GGAGCTCCCCGGCTT,CATGATAGTACAGGCTCCACGTTGG,60,64,63
9,AO090009000373,ACTGACCCAAACTATACACC,ATGAACACTGTAAGTGTTGATACTAAC,56,59,60


In [39]:
gene_names_f = [name+'_f' for name in gene]
gene_names_r = [name+'_r' for name in gene]

In [46]:
forward_df = pd.DataFrame({'name': gene_names_f, 'sequence':forward_primer, 'concentration':'25nm', 'purification': 'STD'})
reverse_df = pd.DataFrame({'name': gene_names_r, 'sequence':r_primer, 'concentration':'25nm', 'purification': 'STD'})

idt_primers_result = pd.concat([forward_df, reverse_df],ignore_index=True)
idt_primers_result


,name,sequence,concentration,purification
0,AO090038000444_f,GCTGGTCCTTGGGCT,25nm,STD
1,AO090012000917_f,CCCCATCCCCGAGTG,25nm,STD
2,AO090005001553_f,TCAGTTTTCCAATGTTAGTCTGC,25nm,STD
3,AO090011000715_f,TCGGGATTCACCTGTAC,25nm,STD
4,AO090011000757_f,CTTCCAACAACCGACAAGA,25nm,STD
5,AO090003000990_f,ACTATGTCAGTATAACCGTGATG,25nm,STD
6,AO090012000006_f,GATGCCGAGGGTTGG,25nm,STD
7,AO090003001590_f,AGTGCCAATTGTACAGGG,25nm,STD
8,AO090005001389_f,GGAGCTCCCCGGCTT,25nm,STD
9,AO090009000373_f,ACTGACCCAAACTATACACC,25nm,STD
